In [1]:
# Preamble
from astropy.table import Table, column, vstack
import numpy as np
import os
import pandas as pd

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

import sys
sys.path.append('../astro_codes/')

In [2]:
gz2_data = Table.read('../fits/full_sample_debiased_w_low_z_mod.fits')

In [3]:
nsa_data = Table.read('../fits/SF_fits/nsa_less.fits')

In [4]:
mendel_data = Table.read('../fits/Simard+Mendel/mendel_dusty.fits')
mendel_data.remove_column('All')

In [5]:
#arc_data = Table.read('panoptes_project/arc_table_ML.fits')
#samples = Table.read('../fits/sparcfire/sparcfire_samples.fits')
#sfr_data = Table.read('../fits/SF_fits/sfrs.fits')
#axial_ratios = Table.read('../fits/Axial_ratios/dr7_isoAB_matched.fits')

#### First task: re-match the Mendel data

In [6]:
from combining_catalogues import match_ids, make_matched_catalogue

match_ids = match_ids(gz2_data,mendel_data,'dr7objid','objID')
mendel_matched = make_matched_catalogue(gz2_data,mendel_data,match_ids)
mendel_matched.write('../fits/Simard+Mendel/mendel_matched.fits',
                     overwrite=True)

248883 galaxies in the reference catalogue
657996 galaxies in the match catalogue
---> 237996 matches in total


In [8]:
mendel_masses = Table()
mendel_masses['dr7id'] = gz2_data['dr7objid']
mendel_masses['dr8id'] = gz2_data['dr8objid']
mendel_masses['total'] = mendel_matched['logM']
mendel_masses['total_lower'] = mendel_matched['b_logM']
mendel_masses['total_upper'] = mendel_matched['b_logm_lc']
mendel_masses['bulge'] = mendel_matched['logMb']
mendel_masses['bulge_lower'] = mendel_matched['b_logMb']
mendel_masses['bulge_upper'] = mendel_matched['b_logmb_lc']
mendel_masses['disc'] = mendel_matched['logMd']
mendel_masses['disc_lower'] = mendel_matched['b_logMd']
mendel_masses['disc_upper'] = mendel_matched['b_logmd_lc']
mendel_masses['bulge+disc'] = mendel_matched['logMt']
mendel_masses['bulge+disc_lower'] = mendel_matched['b_logMt']
mendel_masses['bulge+disc_upper'] = mendel_matched['b_logmt_lc']
mendel_masses['F-test'] = mendel_matched['PpS']

mendel_masses.write('../fits/Simard+Mendel/mendel_masses.fits',
                    overwrite=True)

#### What kind of sample to we require?

* Redshift limited (0.03 < z < 0.055)
* Luminosity-limited?

In [ ]:
from luminosities_magnitudes_and_distances import mag_to_Mag

low_z_limit = 0.02

samples = Table()

in_z_dr7 = np.all([gz2_data['REDSHIFT_1'] >= low_z_limit,
                   gz2_data['REDSHIFT_1'] <= 0.055],axis=0)

in_z_nsa = nsa_data['Z'] >= low_z_limit

print('--- z-limiting --- \n NSA:{} \n DR7:{}'.format(in_z_dr7.sum(),
                                                      in_z_nsa.sum()))

Mr_limit = mag_to_Mag(17,0.055)
in_Mr_dr7 = gz2_data['PETROMAG_MR'] <= Mr_limit
in_Mr_nsa = nsa_data['M_corrected'][:,4] <= Mr_limit
in_volume_limit_dr7 = np.all([in_z_dr7,in_Mr_dr7],axis=0)
in_volume_limit_nsa = np.all([in_z_nsa,in_Mr_nsa],axis=0)

print('--- M_r-limiting --- \n M_r={} \n NSA:{} \n DR7:{}'.format(Mr_limit,
                      in_volume_limit_dr7.sum(),in_volume_limit_nsa.sum()))

low_mass_limit = 10.108576202851912
high_mass_limit = 11.338759782465708

in_mass_limit = np.all([mendel_masses['bulge+disc'] >= low_mass_limit,
                        mendel_masses['bulge+disc'] <= high_mass_limit],
                        axis=0)

in_mass_limit_dr7 = np.all([in_z_dr7,in_Mr_dr7,in_mass_limit],axis=0)
in_mass_limit_nsa = np.all([in_z_nsa,in_Mr_nsa,in_mass_limit],axis=0)
print('--- M*-limiting --- \n NSA:{}, DR7:{}'.format(in_mass_limit_dr7.sum(),
                                                     in_mass_limit_nsa.sum()))

In [ ]:
# Ok, first question: why are only 4000/21000 classed as spirals?

def Np_spiral(gz2_data):
    p_spiral = (gz2_data['t01_smooth_or_features_a02_features_or_disk_debiased_rh']*
                gz2_data['t02_edgeon_a05_no_debiased_rh']*
                gz2_data['t04_spiral_a08_spiral_debiased_rh'])
    
    N_spiral = (gz2_data['t04_spiral_a08_spiral_count'] - 
                gz2_data['t11_arms_number_a37_cant_tell_count'])
    
    p_spiral[np.isfinite(p_spiral) == False] = 0
    N_spiral[np.isfinite(N_spiral) == False] = 0
    
    return p_spiral, N_spiral

In [ ]:
p, N = Np_spiral(gz2_data)

in_spiral = (p >= 0.5) & (N >= 5)

axial_ratio = axial_ratios['isoB_g']/axial_ratios['isoA_g']
axial_ok = axial_ratio >= 0.4

ml_spirals_dr7 = np.all([in_mass_limit_dr7,in_spiral,axial_ok],axis=0)
ml_spirals_nsa = np.all([in_mass_limit_nsa,in_spiral,axial_ok],axis=0)

In [ ]:
# maybe check the axial ratios w. images?
from display_image import display_image
np.random.seed(0)

fig, axarr = plt.subplots(4,5,figsize=(20,16))
axarr = axarr.ravel()

plt.hist(axial_ratio)

axial_bins = np.linspace(0.2,1,21)
for ax, a_min, a_max in zip(axarr,axial_bins[:-1],axial_bins[1:]):
    in_bin = np.all([axial_ratio >= a_min,axial_ratio <= a_max],axis=0)
    jpeg_url = np.random.choice(gz2_data['jpeg_url'][in_bin])
    plt.sca(ax)
    display_image(jpeg_url)

In [ ]:
samples = Table()
samples['dr7id'] = gz2_data['dr7objid']
samples['dr8id'] = gz2_data['dr8objid']
samples['in_z_nsa'] = in_z_nsa
samples['in_vl_nsa'] = in_volume_limit_nsa
samples['in_ml_nsa'] = in_mass_limit_nsa
samples['in_z_dr7'] = in_z_dr7
samples['in_vl_dr7'] = in_volume_limit_dr7
samples['in_ml_dr7'] = in_mass_limit_dr7
samples['in_spiral'] = in_spiral
samples['axial_ratio'] = axial_ratio
samples['axial_ok'] = axial_ok
samples.write('../fits/sparcfire/sparcfire_samples.fits',overwrite=True)